# Creating your own dataset

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!apt install git-lfs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


You will need to setup git, adapt your email and name in the following cell.

In [2]:
!git config --global user.email "you@example.com"
!git config --global user.name "Your Name"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [83]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
!pip install requests

In [5]:
import requests

url = "https://api.github.com/repos/huggingface/datasets/issues?page=1&per_page=1"
response = requests.get(url)

In [6]:
response.status_code

200

In [7]:
response.json()

[{'url': 'https://api.github.com/repos/huggingface/datasets/issues/7951',
  'repository_url': 'https://api.github.com/repos/huggingface/datasets',
  'labels_url': 'https://api.github.com/repos/huggingface/datasets/issues/7951/labels{/name}',
  'comments_url': 'https://api.github.com/repos/huggingface/datasets/issues/7951/comments',
  'events_url': 'https://api.github.com/repos/huggingface/datasets/issues/7951/events',
  'html_url': 'https://github.com/huggingface/datasets/pull/7951',
  'id': 3827686741,
  'node_id': 'PR_kwDODunzps6951bw',
  'number': 7951,
  'title': 'feat: Add GenBank file format support for biological sequence data',
  'user': {'login': 'behroozazarkhalili',
   'id': 80390531,
   'node_id': 'MDQ6VXNlcjgwMzkwNTMx',
   'avatar_url': 'https://avatars.githubusercontent.com/u/80390531?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/behroozazarkhalili',
   'html_url': 'https://github.com/behroozazarkhalili',
   'followers_url': 'https://api.github.com/u

In [6]:
GITHUB_TOKEN = ""  # Copy your GitHub token here
headers = {"Authorization": f"token {GITHUB_TOKEN}"}

In [7]:
import time
import math
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm


def fetch_issues(
    owner="huggingface",
    repo="datasets",
    num_issues=10_000,
    rate_limit=5_000,
    issues_path=Path("."),
):
    if not issues_path.is_dir():
        issues_path.mkdir(exist_ok=True)

    batch = []
    all_issues = []
    per_page = 100  # Number of issues to return per page
    num_pages = math.ceil(num_issues / per_page)
    base_url = "https://api.github.com/repos"

    for page in tqdm(range(num_pages)):
        # Query with state=all to get both open and closed issues
        query = f"issues?page={page}&per_page={per_page}&state=all"
        issues = requests.get(f"{base_url}/{owner}/{repo}/{query}", headers=headers)
        batch.extend(issues.json())

        if len(batch) > rate_limit and len(all_issues) < num_issues:
            all_issues.extend(batch)
            batch = []  # Flush batch for next time period
            print(f"Reached GitHub rate limit. Sleeping for one hour ...")
            time.sleep(60 * 60 + 1)

    all_issues.extend(batch)
    df = pd.DataFrame.from_records(all_issues)
    df.to_json(f"{issues_path}/{repo}-issues.jsonl", orient="records", lines=True)
    print(
        f"Downloaded all the issues for {repo}! Dataset stored at {issues_path}/{repo}-issues.jsonl"
    )

In [ ]:
# Depending on your internet connection, this can take several minutes to run...
fetch_issues()

In [34]:
df = pd.read_json("datasets-issues.jsonl", lines=True)
df.head()

,url,repository_url,labels_url,comments_url,events_url,html_url,id,node_id,number,title,...,draft,pull_request,body,closed_by,reactions,timeline_url,performed_via_github_app,state_reason,sub_issues_summary,issue_dependencies_summary
0,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datasets,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datas...,https://github.com/huggingface/datasets/pull/7953,3831099841,PR_kwDODunzps6-FI-G,7953,#5354: replace list with Sequence in from_parq...,...,0.0,{'url': 'https://api.github.com/repos/huggingf...,\This PR replaces `list` type hints with `Sequ...,None,{'url': 'https://api.github.com/repos/huggingf...,https://api.github.com/repos/huggingface/datas...,NaN,None,None,None
1,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datasets,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datas...,https://github.com/huggingface/datasets/pull/7952,3831024005,PR_kwDODunzps6-E4dA,7952,Fix #5354: replace list with Sequence in from_...,...,0.0,{'url': 'https://api.github.com/repos/huggingf...,This PR replaces `list` type hints with `Seque...,"{'login': 'ashmi8', 'id': 105732253, 'node_id'...",{'url': 'https://api.github.com/repos/huggingf...,https://api.github.com/repos/huggingface/datas...,NaN,None,None,None
2,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datasets,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datas...,https://github.com/huggingface/datasets/pull/7951,3827686741,PR_kwDODunzps6951bw,7951,feat: Add GenBank file format support for biol...,...,0.0,{'url': 'https://api.github.com/repos/huggingf...,## Summary\nAdd native support for loading Gen...,None,{'url': 'https://api.github.com/repos/huggingf...,https://api.github.com/repos/huggingface/datas...,NaN,None,None,None
3,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datasets,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datas...,https://github.com/huggingface/datasets/pull/7950,3825689242,PR_kwDODunzps69zsW1,7950,Add examples for Lance datasets,...,0.0,{'url': 'https://api.github.com/repos/huggingf...,## Summary\r\n\r\nUpdated the Lance integratio...,None,{'url': 'https://api.github.com/repos/huggingf...,https://api.github.com/repos/huggingface/datas...,NaN,None,None,None
4,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datasets,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datas...,https://github.com/huggingface/datasets/pull/7949,3824515306,PR_kwDODunzps69v3lh,7949,docs: clarify documentation build instructions,...,0.0,{'url': 'https://api.github.com/repos/huggingf...,docs: clarify documentation build instructions,None,{'url': 'https://api.github.com/repos/huggingf...,https://api.github.com/repos/huggingface/datas...,NaN,None,None,None


In [37]:
df.columns

Index(['url', 'repository_url', 'labels_url', 'comments_url', 'events_url',
       'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels',
       'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments',
       'created_at', 'updated_at', 'closed_at', 'author_association', 'type',
       'active_lock_reason', 'draft', 'pull_request', 'body', 'closed_by',
       'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason',
       'sub_issues_summary', 'issue_dependencies_summary'],
      dtype='object')

In [40]:
df1=df.drop(columns=['created_at', 'updated_at', 'closed_at']).head()

In [49]:
df1.columns


Index(['url', 'repository_url', 'labels_url', 'comments_url', 'events_url',
       'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels',
       'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments',
       'author_association', 'type', 'active_lock_reason', 'draft',
       'pull_request', 'body', 'closed_by', 'reactions', 'timeline_url',
       'performed_via_github_app', 'state_reason', 'sub_issues_summary',
       'issue_dependencies_summary'],
      dtype='object')

In [68]:
df2 = df1[["html_url", "pull_request", "number"]]
df2

,html_url,pull_request,number
0,https://github.com/huggingface/datasets/pull/7953,{'url': 'https://api.github.com/repos/huggingf...,7953
1,https://github.com/huggingface/datasets/pull/7952,{'url': 'https://api.github.com/repos/huggingf...,7952
2,https://github.com/huggingface/datasets/pull/7951,{'url': 'https://api.github.com/repos/huggingf...,7951
3,https://github.com/huggingface/datasets/pull/7950,{'url': 'https://api.github.com/repos/huggingf...,7950
4,https://github.com/huggingface/datasets/pull/7949,{'url': 'https://api.github.com/repos/huggingf...,7949


In [69]:
df2.to_json("datasets-issues-modified.jsonl", orient="records", lines=True)
print("DataFrame saved to datasets-issues-modified.jsonl")

DataFrame saved to datasets-issues-modified.jsonl


In [ ]:
issues_dataset = load_dataset("json", data_files="datasets-issues-modified.jsonl", split="train")
issues_dataset

In [71]:
sample = issues_dataset.shuffle(seed=666).select(range(3))

# Print out the URL and pull request entries
for url, pr in zip(sample["html_url"], sample["pull_request"]):
    print(f">> URL: {url}")
    print(f">> Pull request: {pr}\n")

>> URL: https://github.com/huggingface/datasets/pull/7953
>> Pull request: {'url': 'https://api.github.com/repos/huggingface/datasets/pulls/7953', 'html_url': 'https://github.com/huggingface/datasets/pull/7953', 'diff_url': 'https://github.com/huggingface/datasets/pull/7953.diff', 'patch_url': 'https://github.com/huggingface/datasets/pull/7953.patch', 'merged_at': None}

>> URL: https://github.com/huggingface/datasets/pull/7950
>> Pull request: {'url': 'https://api.github.com/repos/huggingface/datasets/pulls/7950', 'html_url': 'https://github.com/huggingface/datasets/pull/7950', 'diff_url': 'https://github.com/huggingface/datasets/pull/7950.diff', 'patch_url': 'https://github.com/huggingface/datasets/pull/7950.patch', 'merged_at': None}

>> URL: https://github.com/huggingface/datasets/pull/7951
>> Pull request: {'url': 'https://api.github.com/repos/huggingface/datasets/pulls/7951', 'html_url': 'https://github.com/huggingface/datasets/pull/7951', 'diff_url': 'https://github.com/huggingf

In [ ]:
issues_dataset = issues_dataset.map(
    lambda x: {"is_pull_request": False if x["pull_request"] is None else True}
)

In [73]:
issue_number = 2792
url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
response = requests.get(url, headers=headers)
response.json()

[{'url': 'https://api.github.com/repos/huggingface/datasets/issues/comments/897594128',
  'html_url': 'https://github.com/huggingface/datasets/pull/2792#issuecomment-897594128',
  'issue_url': 'https://api.github.com/repos/huggingface/datasets/issues/2792',
  'id': 897594128,
  'node_id': 'IC_kwDODunzps41gDMQ',
  'user': {'login': 'bhavitvyamalik',
   'id': 19718818,
   'node_id': 'MDQ6VXNlcjE5NzE4ODE4',
   'avatar_url': 'https://avatars.githubusercontent.com/u/19718818?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/bhavitvyamalik',
   'html_url': 'https://github.com/bhavitvyamalik',
   'followers_url': 'https://api.github.com/users/bhavitvyamalik/followers',
   'following_url': 'https://api.github.com/users/bhavitvyamalik/following{/other_user}',
   'gists_url': 'https://api.github.com/users/bhavitvyamalik/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/bhavitvyamalik/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/

In [74]:
def get_comments(issue_number):
    url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
    response = requests.get(url, headers=headers)
    return [r["body"] for r in response.json()]


# Test our function works as expected
get_comments(2792)

["@albertvillanova my tests are failing here:\r\n```\r\ndataset_name = 'gooaq'\r\n\r\n    def test_load_dataset(self, dataset_name):\r\n        configs = self.dataset_tester.load_all_configs(dataset_name, is_local=True)[:1]\r\n>       self.dataset_tester.check_load_dataset(dataset_name, configs, is_local=True, use_local_dummy_data=True)\r\n\r\ntests/test_dataset_common.py:234: \r\n_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ \r\ntests/test_dataset_common.py:187: in check_load_dataset\r\n    self.parent.assertTrue(len(dataset[split]) > 0)\r\nE   AssertionError: False is not true\r\n```\r\nWhen I try loading dataset on local machine it works fine. Any suggestions on how can I avoid this error?",
 'Thanks for the help, @albertvillanova! All tests are passing now.']

In [ ]:
# Depending on your internet connection, this can take a few minutes...
issues_with_comments_dataset = issues_dataset.map(
    lambda x: {"comments": get_comments(x["number"])}
)

In [89]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
issues_with_comments_dataset.push_to_hub("github-issues")

In [91]:
remote_dataset = load_dataset("lewtun/github-issues", split="train")
remote_dataset

Repo card metadata block was not found. Setting CardData to empty.


Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 3019
})